In [31]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import time

In [32]:
url_last = 'https://etfdb.com/esg-investing/environmental-issues/'

In [52]:
def click_reader_new(url= url_last, num_pages = 97, click_button = "Next ›", verbose = True):
    # collect each page into a list
    dataframes = []

    # chrome session
    chrome_options = Options()
    chrome_options.add_argument('--headless')
    chrome_options.add_argument('--disable-gpu')  # Only necessary if you run on Windows

    driver = webdriver.Chrome(options = chrome_options)
    driver.get(url)

    # Custom wait condition for handling stale elements
    def staleness_of_element(locator):
        def predicate(driver):
            try:
                element = driver.find_element(*locator)
                return not element.is_displayed()
            except Exception:
                return True
        return predicate


    # Using partial link text to locate the button with text "2"
    button_locator = (By.PARTIAL_LINK_TEXT, click_button)

    # def click_element_safely(locator):
    #     attempts = 0
    #     while attempts < 3:
    #         try:
    #             button = WebDriverWait(driver, 20).until(EC.element_to_be_clickable(locator))
    #             button.click()
    #             return True
    #         except Exception as e:
    #             print(f"Exception occurred: {e}")
    #             driver.refresh()
    #             # Wait for the element to become stale
    #             WebDriverWait(driver, 20).until(staleness_of_element(locator))
    #             attempts += 1
    #     return False

    def click_element_safely(locator):
        attempts = 0
        while attempts < 3:
            try:
                button = WebDriverWait(driver, 30).until(EC.element_to_be_clickable(locator))
                time.sleep(1)  # Additional wait to ensure element is ready
                # Click using JavaScript to bypass overlapping elements
                driver.execute_script("arguments[0].click();", button)
                return True
            except Exception as e:
                print(f"Exception occurred: {e}")
                driver.refresh()
                # Wait for the element to become stale
                WebDriverWait(driver, 30).until(staleness_of_element(locator))
                attempts += 1
        return False

    n = 1
    df_list = pd.read_html(url_last)
    df = df_list[0]
    dataframes =  dataframes + [df.iloc[range(df.shape[0]-1),:]]
    
    while n < (num_pages+1):
        time.sleep(3)
        if verbose:
            print(n+1)
        # click the next button
        click_element_safely(button_locator)
        # Get the page source after JavaScript rendering
        html_content = driver.page_source
        # Convert the rendered HTML content to DataFrame
        df_list2 = pd.read_html(html_content)
        if df_list2:
            df = df_list2[0]  # Select the first table if multiple tables exist on the page
        dataframes =  dataframes + [df.iloc[range(df.shape[0]-1),:]]

        # to read next page
        n = n+1
       
    driver.quit()  
    # combine all pages
    return pd.concat(dataframes)

In [53]:
result_new = click_reader_new()

2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98


In [54]:
result_new.tail()

,Symbol Symbol,ETF Name ETF Name,Total Assets* Total Assets*,YTD YTD,Avg Volume Avg Volume,Previous Closing Price Previous Closing Price,1-Day Change 1-Day Change,1 Week 1 Week,1 Month 1 Month,1 Year 1 Year,...,Dividend Rating Dividend Rating,Concentration Rating Concentration Rating,ESG Score ESG Score,ESG Score Peer Percentile (%) ESG Score Peer Percentile (%),ESG Score Global Percentile (%) ESG Score Global Percentile (%),Weighted Average Carbon Intensity (Tons of CO2e / $M Sales) Weighted Average Carbon Intensity (Tons of CO2e / $M Sales),SRI Exclusion Criteria (%) SRI Exclusion Criteria (%),Sustainable Impact Solutions (%) Sustainable Impact Solutions (%),Revenue Exposure to Environmental Impact (%) Revenue Exposure to Environmental Impact (%),Severe Environment Controversies (%) Severe Environment Controversies (%)
3,XTRE,BondBloxx Bloomberg Three Year Target Duration...,"$27,385.30",0.96%,2668.0,$48.74,-0.33%,-0.40%,-0.50%,NaN,...,NaN,NaN,5.72,12.64%,34.34%,0.0,0.00%,0.00%,NaN,NaN
4,XTWO,BondBloxx Bloomberg Two Year Target Duration U...,"$92,275.90",1.11%,19210.0,$49.01,-0.16%,-0.18%,-0.13%,NaN,...,NaN,NaN,5.72,87.04%,34.34%,0.0,0.00%,0.00%,NaN,NaN
5,XTWY,BondBloxx Bloomberg Twenty Year Target Duratio...,"$30,916.80",1.18%,4497.0,$45.17,-2.08%,-2.61%,-4.04%,NaN,...,NaN,NaN,5.72,79.05%,34.34%,0.0,0.00%,0.00%,NaN,NaN
6,YANG,Direxion Daily FTSE China Bear 3X Shares,"$123,870.00",-20.13%,3390829.0,$9.30,4.26%,-9.80%,-16.59%,-26.84%,...,NaN,NaN,5.32,27.42%,28.14%,1.6,0.00%,0.00%,NaN,NaN
7,ZROZ,PIMCO 25+ Year Zero Coupon US Treasury Index ETF,"$918,686.00",0.89%,225190.0,$86.69,-2.55%,-3.38%,-5.15%,-18.80%,...,NaN,NaN,5.72,79.05%,34.34%,0.0,0.00%,0.00%,NaN,NaN


In [56]:
result_new.columns = [a[0:int((len(a)-1)/2)] for a in result_new.columns]
result_new.iloc[:5,:]

,Symbol,ETF Name,Total Assets*,YTD,Avg Volume,Previous Closing Price,1-Day Change,1 Week,1 Month,1 Year,...,Dividend Rating,Concentration Rating,ESG Score,ESG Score Peer Percentile (%),ESG Score Global Percentile (%),Weighted Average Carbon Intensity (Tons of CO2e / $M Sales),SRI Exclusion Criteria (%),Sustainable Impact Solutions (%),Revenue Exposure to Environmental Impact (%),Severe Environment Controversies (%)
0,ERTH,Invesco MSCI Sustainable Future ETF,"$291,838.00",8.13%,23768.0,$51.36,-2.58%,0.14%,9.14%,-7.11%,...,NaN,NaN,7.05,76.40%,73.12%,144.59,0.04%,77.54%,NaN,NaN
1,WNDY,Global X Wind Energy ETF,"$4,654.58",-11.89%,4089.0,$15.68,-2.12%,-1.72%,-0.26%,-20.31%,...,NaN,NaN,7.06,68.72%,73.70%,196.35,0.00%,72.37%,NaN,NaN
2,FRNW,Fidelity Clean Energy ETF,"$39,006.50",-5.52%,5678.0,$20.44,-3.45%,-3.80%,0.78%,-8.13%,...,NaN,NaN,7.23,73.13%,79.87%,321.5,0.00%,68.67%,NaN,NaN
3,KGRN,KraneShares MSCI China Clean Technology Index ETF,"$90,537.90",7.32%,17816.0,$29.18,-0.78%,6.57%,10.11%,-22.08%,...,NaN,NaN,6.06,97.98%,40.52%,350.0,0.00%,68.01%,NaN,NaN
4,ACES,ALPS Clean Energy ETF,"$508,405.00",3.25%,81129.0,$46.76,-2.77%,-4.23%,10.14%,-11.35%,...,NaN,NaN,6.96,65.64%,69.73%,173.65,0.00%,66.68%,NaN,NaN


In [57]:
result_new.tail()

,Symbol,ETF Name,Total Assets*,YTD,Avg Volume,Previous Closing Price,1-Day Change,1 Week,1 Month,1 Year,...,Dividend Rating,Concentration Rating,ESG Score,ESG Score Peer Percentile (%),ESG Score Global Percentile (%),Weighted Average Carbon Intensity (Tons of CO2e / $M Sales),SRI Exclusion Criteria (%),Sustainable Impact Solutions (%),Revenue Exposure to Environmental Impact (%),Severe Environment Controversies (%)
3,XTRE,BondBloxx Bloomberg Three Year Target Duration...,"$27,385.30",0.96%,2668.0,$48.74,-0.33%,-0.40%,-0.50%,NaN,...,NaN,NaN,5.72,12.64%,34.34%,0.0,0.00%,0.00%,NaN,NaN
4,XTWO,BondBloxx Bloomberg Two Year Target Duration U...,"$92,275.90",1.11%,19210.0,$49.01,-0.16%,-0.18%,-0.13%,NaN,...,NaN,NaN,5.72,87.04%,34.34%,0.0,0.00%,0.00%,NaN,NaN
5,XTWY,BondBloxx Bloomberg Twenty Year Target Duratio...,"$30,916.80",1.18%,4497.0,$45.17,-2.08%,-2.61%,-4.04%,NaN,...,NaN,NaN,5.72,79.05%,34.34%,0.0,0.00%,0.00%,NaN,NaN
6,YANG,Direxion Daily FTSE China Bear 3X Shares,"$123,870.00",-20.13%,3390829.0,$9.30,4.26%,-9.80%,-16.59%,-26.84%,...,NaN,NaN,5.32,27.42%,28.14%,1.6,0.00%,0.00%,NaN,NaN
7,ZROZ,PIMCO 25+ Year Zero Coupon US Treasury Index ETF,"$918,686.00",0.89%,225190.0,$86.69,-2.55%,-3.38%,-5.15%,-18.80%,...,NaN,NaN,5.72,79.05%,34.34%,0.0,0.00%,0.00%,NaN,NaN


In [66]:
result_new_ind = result_new.reset_index()

In [68]:
result_new_ind_2 = result_new_ind.reset_index()

In [71]:
result_new_ind_2.drop(columns='index',inplace=True)

In [75]:
result_new_ind_2.rename(columns={'level_0':'Index'},inplace=True)

In [76]:
result_new_ind_2.head()

,Index,Symbol,ETF Name,Total Assets*,YTD,Avg Volume,Previous Closing Price,1-Day Change,1 Week,1 Month,...,Dividend Rating,Concentration Rating,ESG Score,ESG Score Peer Percentile (%),ESG Score Global Percentile (%),Weighted Average Carbon Intensity (Tons of CO2e / $M Sales),SRI Exclusion Criteria (%),Sustainable Impact Solutions (%),Revenue Exposure to Environmental Impact (%),Severe Environment Controversies (%)
0,0,ERTH,Invesco MSCI Sustainable Future ETF,"$291,838.00",8.13%,23768.0,$51.36,-2.58%,0.14%,9.14%,...,NaN,NaN,7.05,76.40%,73.12%,144.59,0.04%,77.54%,NaN,NaN
1,1,WNDY,Global X Wind Energy ETF,"$4,654.58",-11.89%,4089.0,$15.68,-2.12%,-1.72%,-0.26%,...,NaN,NaN,7.06,68.72%,73.70%,196.35,0.00%,72.37%,NaN,NaN
2,2,FRNW,Fidelity Clean Energy ETF,"$39,006.50",-5.52%,5678.0,$20.44,-3.45%,-3.80%,0.78%,...,NaN,NaN,7.23,73.13%,79.87%,321.5,0.00%,68.67%,NaN,NaN
3,3,KGRN,KraneShares MSCI China Clean Technology Index ETF,"$90,537.90",7.32%,17816.0,$29.18,-0.78%,6.57%,10.11%,...,NaN,NaN,6.06,97.98%,40.52%,350.0,0.00%,68.01%,NaN,NaN
4,4,ACES,ALPS Clean Energy ETF,"$508,405.00",3.25%,81129.0,$46.76,-2.77%,-4.23%,10.14%,...,NaN,NaN,6.96,65.64%,69.73%,173.65,0.00%,66.68%,NaN,NaN


In [77]:
result_new_ind_2.columns = result_new_ind_2.columns.str.strip()
result_new_ind_2.columns = result_new_ind_2.columns.str.replace(' ','_')
result_new_ind_2.columns = result_new_ind_2.columns.str.lower()


In [80]:
result_new_ind_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2433 entries, 0 to 2432
Data columns (total 61 columns):
 #   Column                                                       Non-Null Count  Dtype 
---  ------                                                       --------------  ----- 
 0   index                                                        2433 non-null   int64 
 1   symbol                                                       2433 non-null   object
 2   etf_name                                                     2433 non-null   object
 3   total_assets*                                                2432 non-null   object
 4   ytd                                                          2331 non-null   object
 5   avg_volume                                                   2393 non-null   object
 6   previous_closing_price                                       2406 non-null   object
 7   1-day_change                                                 2433 non-null   object
 8 

In [90]:
result_new_ind_2.isna().sum()

index                                                             0
symbol                                                            0
etf_name                                                          0
total_assets*                                                     1
ytd                                                             102
                                                               ... 
weighted_average_carbon_intensity_(tons_of_co2e_/_$m_sales)       0
sri_exclusion_criteria_(%)                                        0
sustainable_impact_solutions_(%)                                  0
revenue_exposure_to_environmental_impact_(%)                   2433
severe_environment_controversies_(%)                           2433
Length: 61, dtype: int64

In [106]:
result_new_ind_2.isnull().sum().sort_values(ascending=False)

1_year_ff                                                      2433
returns_rating                                                 2433
volatility_rating                                              2433
dividend_rating                                                2433
concentration_rating                                           2433
overall_rating                                                 2433
5_year_ff                                                      2433
3_year_ff                                                      2433
ytd_ff                                                         2433
1_week_ff                                                      2433
4_week_ff                                                      2433
commission_free                                                2098
5_year                                                         1071
3_year                                                          797
1_year                                          

In [104]:
result_new_ind_2.drop(columns=['revenue_exposure_to_environmental_impact_(%)','severe_environment_controversies_(%)'],inplace=True)

In [107]:
dropped_columns = ['1_year_ff',
                   'returns_rating',
                   'volatility_rating',
                   'dividend_rating',
                   'concentration_rating',
                   'overall_rating',
                   '5_year_ff',
                   '3_year_ff',
                   'ytd_ff',
                   '1_week_ff',
                   '4_week_ff',
                   'commission_free',
                   '5_year']                                         
# 1_year_ff                                                      2433
# returns_rating                                                 2433
# volatility_rating                                              2433
# dividend_rating                                                2433
# concentration_rating                                           2433
# overall_rating                                                 2433
# 5_year_ff                                                      2433
# 3_year_ff                                                      2433
# ytd_ff                                                         2433
# 1_week_ff                                                      2433
# 4_week_ff                                                      2433
# commission_free                                                2098
# 5_year                                                         1071

In [108]:
result_new_ind_2.drop(columns=dropped_columns,inplace=True)

In [109]:
result_new_ind_2.isnull().sum().sort_values(ascending=False)

3_year                                                         797
1_year                                                         245
etf_database_category                                          231
liquidity_rating                                               216
dividend                                                       121
dividend_date                                                  121
ytd                                                            102
support_1                                                       84
resistance_1                                                    84
beta                                                            68
p/e_ratio                                                       68
avg_volume                                                      40
annual_dividend_rate                                            28
annual_dividend_yield_%                                         28
upper_bollinger                                               

In [111]:
result_new_ind_2.shape

(2433, 46)

In [92]:
result_new_ind_2.duplicated().sum()

0

In [93]:
result_new_ind_2.shape

(2433, 61)

In [99]:
result_new_ind_2.describe()

,index
count,2433.000000
mean,1216.000000
std,702.490925
min,0.000000
25%,608.000000
50%,1216.000000
75%,1824.000000
max,2432.000000


In [101]:
result_new_ind_2.dtypes

index                                                           int64
symbol                                                         object
etf_name                                                       object
total_assets*                                                  object
ytd                                                            object
                                                                ...  
weighted_average_carbon_intensity_(tons_of_co2e_/_$m_sales)    object
sri_exclusion_criteria_(%)                                     object
sustainable_impact_solutions_(%)                               object
revenue_exposure_to_environmental_impact_(%)                   object
severe_environment_controversies_(%)                           object
Length: 61, dtype: object

In [112]:
result_new_ind_2.to_csv('etfdb_97_pages_last.csv')